In [ ]:
! pip install transformers

     |████████████████████████████████| 3.3 MB 9.2 MB/s 
     |████████████████████████████████| 3.3 MB 36.7 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 61 kB 326 kB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm_notebook as tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, BertTokenizerFast
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
name_model = "cointegrated/LaBSE-en-ru"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/NLP_project/Data/Dataset_with_target.csv')

In [ ]:
df.head()

,question_text,answer_text,target
0,Добрый день! Мы бы хотели пройти бесплатное о...,Здравствуйте! Регистрируйтесь на сайте https:/...,0
1,Добрый день.Помогите пожалуйста пройти регистр...,Добрый вечер!В системе на Вашу почту зарегистр...,0
2,Здравствуйте. Начала изучать курс для учителя ...,Здравствуйте! Возможно Ваша презентация имеет ...,0
3,"13 лет,интересует стоимость",Здравствуйте! Курс бесплатный -- можете регист...,0
4,"Здравствуйте, подскажите, пожалуйста, я прошёл...","Здравствуйте! Сертификат придет Вам на почту, ...",0


In [ ]:
df_train, tmp = train_test_split(df, random_state=42,  stratify=df['target'], test_size=0.3)
df_val, df_test = train_test_split(tmp, random_state=42,  stratify=tmp['target'], test_size=0.5)

# Train model

In [ ]:
class model_dataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['question_text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        return_token_type_ids=False,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']      
        target = self.df['target'].values[index]
        
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long).flatten(),
            'attention_mask': torch.tensor(mask, dtype=torch.long).flatten(),
            'target': torch.tensor(target, dtype=torch.long)
        }  

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(name_model)

train_dataset = model_dataset(df_train, tokenizer=tokenizer, max_length=300)
val_dataset = model_dataset(df_val, tokenizer=tokenizer, max_length=300)

Downloading:   0%|          | 0.00/509k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/806 [00:00<?, ?B/s]

Work with a dataloader

In [ ]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) 
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

Classification

In [ ]:
class QuestionsClassifier(nn.Module):
    def __init__(self, n_classes):
        super(QuestionsClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(name_model)
        self.drop = nn.Dropout(p=0.2)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        output = self.drop(outputs['last_hidden_state'][:,0,:])
        return self.out(output)

In [ ]:
n_class = len(df_train['target'].unique())
print(f'n_class: {n_class}')
bert_model = QuestionsClassifier(n_class)

n_class: 2


Downloading:   0%|          | 0.00/492M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import gc
def cleanup():
    """
    A function that clears the cache on CUDA
    """
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [ ]:
bert_model.to(device);

Prepare for fine-tune our model

In [ ]:
from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(bert_model.parameters(),
                  lr = 2e-5 # args.learning_rate - default is 5e-5
                )

epochs = 11
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
loss_fn = nn.CrossEntropyLoss().to(device)

Function to train 1 epoch

In [ ]:
def train_epoch(model, dataset, loss_fn,
                optimizer, device, scheduler, n_examples):
    batch_size = 32
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True) 

    model = model.train()

    losses = []
    correct_predictions = 0

    for i, d in enumerate(tqdm(data_loader)):
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        targets = d['target'].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask)
        
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if i % 100 == 0:
            cleanup()
    cleanup()
    return correct_predictions.double() / n_examples, np.mean(losses) #Accuracy and Loss

Function to evaluate our model

In [ ]:
def eval_model(model, dataset, loss_fn, device, n_examples):
    batch_size = 32
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True) 

    model = model.eval()

    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for i, d in enumerate(tqdm(data_loader)):
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            targets = d['target'].to(device)

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask)
            
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
            if i % 3000 == 0:
                cleanup()
        cleanup()

    return correct_predictions.double() / n_examples, np.mean(losses) #Accuracy and Loss



Fine-tune our model

In [ ]:
best_accuaracy = 0

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(model=bert_model, 
                                        dataset=train_dataset, 
                                        loss_fn=loss_fn, 
                                        optimizer=optimizer, 
                                        device=device, 
                                        scheduler=scheduler, 
                                        n_examples=len(df_train))
    
    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(model=bert_model, 
                                    dataset=val_dataset, 
                                    loss_fn=loss_fn,
                                    device=device,
                                    n_examples=len(df_val))
    
    print(f'Val loss:{val_loss}  accuracy:{val_acc}')

    if val_acc > best_accuaracy:
        torch.save(bert_model.state_dict(), 'classification_bert.bin')
        best_accuracy = val_acc



Epoch 1/11
----------


  0%|          | 0/104 [00:00<?, ?it/s]

Train loss 0.39680495256414783 accuracy 0.8626506024096386


  0%|          | 0/23 [00:00<?, ?it/s]

Val loss:0.38904238913370215  accuracy:0.8665730337078651
Epoch 2/11
----------


...

Val loss:1.2352342294610066  accuracy:0.8469101123595505
Epoch 11/11
----------


  0%|          | 0/104 [00:00<?, ?it/s]

Train loss 0.0022798331795526724 accuracy 0.999698795180723


  0%|          | 0/23 [00:00<?, ?it/s]

Val loss:1.2263037268396304  accuracy:0.8497191011235955


# Test model

In [ ]:
name_model = "cointegrated/LaBSE-en-ru"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

device: cuda:0


Load our fine tuned model

In [ ]:
n_class = len(df['target'].unique())
print(f'n_class = {n_class}')
bert_model = QuestionsClassifier(n_class)
tokenizer = BertTokenizerFast.from_pretrained(name_model)
bert_model.load_state_dict(torch.load('/content/drive/MyDrive/NLP/NLP_project/good work/Data/classification_bert1.bin'))
bert_model.eval();

n_class = 2


Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_model.to(device);

In [ ]:
loss_fn = nn.CrossEntropyLoss().to(device)
test_dataset = model_dataset(df_test, tokenizer=tokenizer, max_length=300)

Evaluation of the BERT model on the test

In [ ]:
test_acc, test_loss = eval_model(model=bert_model,
                                 dataset=test_dataset,
                                loss_fn=loss_fn,
                                device=device,
                                n_examples=len(df_test))

print(f'Test loss:{test_acc}  accuracy:{test_acc}')

  0%|          | 0/23 [00:00<?, ?it/s]

Test loss:0.8497191011235955  accuracy:0.8497191011235955
